In [92]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense,Flatten,GlobalAveragePooling2D
import numpy as np
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
import tensorflow as tf

## 1.Loading data

In [151]:
data = pd.read_csv('Datasets/forestfires_nn.csv')
data.head()

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,...,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,...,0,0,0,0,1,0,0,0,0,small
1,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,...,0,0,0,0,0,0,0,1,0,small
2,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,...,0,0,0,0,0,0,0,1,0,small
3,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,...,0,0,0,0,1,0,0,0,0,small
4,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,...,0,0,0,0,1,0,0,0,0,small


## 2.data Analysis

In [4]:
data.shape

(517, 31)

In [5]:
data.dtypes

month             object
day               object
FFMC             float64
DMC              float64
DC               float64
ISI              float64
temp             float64
RH                 int64
wind             float64
rain             float64
area             float64
dayfri             int64
daymon             int64
daysat             int64
daysun             int64
daythu             int64
daytue             int64
daywed             int64
monthapr           int64
monthaug           int64
monthdec           int64
monthfeb           int64
monthjan           int64
monthjul           int64
monthjun           int64
monthmar           int64
monthmay           int64
monthnov           int64
monthoct           int64
monthsep           int64
size_category     object
dtype: object

In [6]:
data.isna().sum()

month            0
day              0
FFMC             0
DMC              0
DC               0
ISI              0
temp             0
RH               0
wind             0
rain             0
area             0
dayfri           0
daymon           0
daysat           0
daysun           0
daythu           0
daytue           0
daywed           0
monthapr         0
monthaug         0
monthdec         0
monthfeb         0
monthjan         0
monthjul         0
monthjun         0
monthmar         0
monthmay         0
monthnov         0
monthoct         0
monthsep         0
size_category    0
dtype: int64

## 3.Data preprocessing

In [152]:
data = data.drop(labels=['month','day'],axis=1)

In [135]:
from sklearn.preprocessing import LabelEncoder

In [153]:
data.loc[data["size_category"]=='small','size_category']=0
data.loc[data["size_category"]=='large','size_category']=1
data["size_category"].value_counts()

0    378
1    139
Name: size_category, dtype: int64

In [154]:
data.head()

,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,dayfri,...,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0,1,...,0,0,0,0,1,0,0,0,0,0
1,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0,0,...,0,0,0,0,0,0,0,1,0,0
2,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0,0,...,0,0,0,0,0,0,0,1,0,0
3,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0,1,...,0,0,0,0,1,0,0,0,0,0
4,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0,0,...,0,0,0,0,1,0,0,0,0,0


In [137]:
data['size_category'].unique()

array([1, 0])

### Seperating X and y

In [155]:
X = data.drop(labels='size_category',axis=1)
y = data['size_category']
X.shape,y.shape

((517, 28), (517,))

### Normalizing X

In [14]:
from sklearn.preprocessing import Normalizer

In [156]:
norm = Normalizer()

col = X.columns
X = pd.DataFrame(norm.fit_transform(X),columns=col)
X.head()

,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,dayfri,...,monthdec,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep
0,0.613364,0.186428,0.671000,0.036290,0.058348,0.362895,0.047674,0.000000,0.0,0.007116,...,0.0,0.0,0.0,0.0,0.0,0.007116,0.0,0.0,0.000000,0.0
1,0.133784,0.052273,0.988020,0.009893,0.026580,0.048729,0.001329,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.001477,0.0
2,0.130323,0.062860,0.988064,0.009638,0.021001,0.047468,0.001870,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.001438,0.0
3,0.578794,0.210184,0.489166,0.056806,0.052388,0.612247,0.025247,0.001262,0.0,0.006312,...,0.0,0.0,0.0,0.0,0.0,0.006312,0.0,0.0,0.000000,0.0
4,0.506542,0.290992,0.579715,0.054455,0.064665,0.561564,0.010210,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.005672,0.0,0.0,0.000000,0.0


### Train test split

In [157]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20,shuffle=True,random_state=37)
X_train.shape,X_test.shape,y_train.shape,y_test.shape


((413, 28), (104, 28), (413,), (104,))

## ANN model building and compiling

In [99]:

model = Sequential()

model.add(Dense(units=5,input_dim = X_train.shape[1],activation = 'relu'))

model.add(Dense(units=5,activation='relu'))

model.add(Dense(1,kernel_initializer='normal',activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['acc'])

## MOdel training

In [105]:
model.fit(X_train,y_train,batch_size=20,epochs=150)

Epoch 1/150
21/21 [==============================] - 0s 1ms/step - loss: 0.2022 - acc: 0.9080
Epoch 2/150
21/21 [==============================] - 0s 1ms/step - loss: 0.2008 - acc: 0.9201
Epoch 3/150
21/21 [==============================] - 0s 1ms/step - loss: 0.1981 - acc: 0.9153
Epoch 4/150
21/21 [==============================] - 0s 1ms/step - loss: 0.1978 - acc: 0.9225
Epoch 5/150
21/21 [==============================] - 0s 968us/step - loss: 0.1959 - acc: 0.9128
Epoch 6/150
21/21 [==============================] - 0s 1ms/step - loss: 0.1919 - acc: 0.9225
Epoch 7/150
21/21 [==============================] - 0s 1ms/step - loss: 0.1916 - acc: 0.9201
Epoch 8/150
21/21 [==============================] - 0s 1ms/step - loss: 0.1915 - acc: 0.9249
Epoch 9/150
21/21 [==============================] - 0s 1ms/step - loss: 0.1872 - acc: 0.9249
Epoch 10/150
21/21 [==============================] - 0s 1ms/step - loss: 0.1853 - acc: 0.9249
Epoch 11/150
21/21 [==============================] - 0s 

21/21 [==============================] - 0s 1ms/step - loss: 0.0900 - acc: 0.9734
Epoch 88/150
21/21 [==============================] - 0s 1ms/step - loss: 0.0891 - acc: 0.9709
Epoch 89/150
21/21 [==============================] - 0s 877us/step - loss: 0.0891 - acc: 0.9734
Epoch 90/150
21/21 [==============================] - 0s 838us/step - loss: 0.0896 - acc: 0.9661
Epoch 91/150
21/21 [==============================] - 0s 819us/step - loss: 0.0879 - acc: 0.9782
Epoch 92/150
21/21 [==============================] - 0s 807us/step - loss: 0.0862 - acc: 0.9734
Epoch 93/150
21/21 [==============================] - 0s 816us/step - loss: 0.0852 - acc: 0.9758
Epoch 94/150
21/21 [==============================] - 0s 812us/step - loss: 0.0847 - acc: 0.9758
Epoch 95/150
21/21 [==============================] - 0s 815us/step - loss: 0.0841 - acc: 0.9734
Epoch 96/150
21/21 [==============================] - 0s 826us/step - loss: 0.0843 - acc: 0.9782
Epoch 97/150
21/21 [===========================

In [158]:
pred_train = model.predict(np.array(X_train))


In [159]:
#Converting the predicted values to series 
pred_train = pd.Series([i[0] for i in pred_train])

size = ["small","large"]
pred_train_class = pd.Series(["small"]*413)
pred_train_class[[i>0.5 for i in pred_train]]= "large"


0      large
1      large
2      large
3      small
4      small
       ...  
408    small
409    large
410    large
411    large
412    large
Length: 413, dtype: object

In [170]:
train = pd.concat([pd.DataFrame(X_train),pd.DataFrame(y_train)],axis=1)

In [171]:
col =data.columns


In [172]:
train.columns = col


In [174]:
#For training data
from sklearn.metrics import confusion_matrix

train["original_class"] = "small"
train.loc[train["size_category"]==1,"original_class"] = "large"
train.original_class.value_counts()
confusion_matrix(pred_train_class,train["original_class"])
print(100-np.mean(pred_train_class==pd.Series(train["original_class"]).reset_index(drop=True)))#100%
pd.crosstab(pred_train_class,pd.Series(train["original_class"]).reset_index(drop=True))

99.98789346246973


original_class,large,small
row_0,,
large,5,303
small,105,0


In [180]:
#For test data
pred_test = model.predict(np.array(X_test))

In [181]:
pred_test = pd.Series([i[0] for i in pred_test])

In [182]:
pred_test.shape

(104,)

In [183]:
pred_test_class = pd.Series(["small"]*104)
pred_test_class[[i>0.5 for i in pred_test]] = "large"
pred_test_class

0      large
1      large
2      large
3      large
4      large
       ...  
99     large
100    large
101    large
102    small
103    large
Length: 104, dtype: object

In [186]:
test =pd.concat([pd.DataFrame(X_test),pd.DataFrame(y_test)],axis=1)
test.head()

,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,dayfri,...,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
436,0.129508,0.291077,0.945790,0.011531,0.037685,0.049216,0.001828,0.0,0.000759,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0
353,0.121304,0.130392,0.981626,0.012644,0.026078,0.061903,0.003556,0.0,0.002265,0.001317,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001317,0
112,0.125066,0.125485,0.982126,0.006693,0.031789,0.054377,0.005019,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001394,0
396,0.118384,0.126494,0.981736,0.014912,0.026685,0.071946,0.006410,0.0,0.004762,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001308,0
269,0.134570,0.222969,0.961718,0.020894,0.027908,0.077440,0.003945,0.0,0.006429,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0


In [187]:
test["original_class"]="small"
test.loc[test["size_category"]==1,"original_class"] = "large"

test["original_class"].value_counts()
np.mean(pred_test_class==pd.Series(test["original_class"]).reset_index(drop=True)) # 85%
confusion_matrix(pred_test_class,test["original_class"])
pd.crosstab(pred_test_class,pd.Series(test["original_class"]).reset_index(drop=True))

original_class,large,small
row_0,,
large,7,75
small,22,0
